In [12]:
import sys
sys.path.append("../MILESTONE1")
from file import df_suffle

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, ReLU, MaxPooling2D,
    GlobalAveragePooling2D, Dense, Dropout
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [14]:
train_df, val_df = train_test_split(
    df_suffle,
    test_size=0.2,
    random_state=42,
    stratify=df_suffle['Brand']
)


In [15]:
IMG_SIZE = 224
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=3,
    brightness_range=[0.9, 1.1]
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)


In [17]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col="Image",
    y_col="Brand",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col="Image",
    y_col="Brand",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=False
)



Found 400 validated image filenames belonging to 3 classes.
Found 100 validated image filenames belonging to 3 classes.


In [19]:
model = Sequential([

    Conv2D(32, (3,3), padding='same', input_shape=(224,224,1)),
    BatchNormalization(),
    ReLU(),
    MaxPooling2D(2),

    Conv2D(64, (3,3), padding='same'),
    BatchNormalization(),
    ReLU(),
    MaxPooling2D(2),

    Conv2D(128, (3,3), padding='same'),
    BatchNormalization(),
    ReLU(),
    MaxPooling2D(2),

    Conv2D(256, (3,3), padding='same'),
    BatchNormalization(),
    ReLU(),

    GlobalAveragePooling2D(),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(3, activation='softmax')
])


In [20]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [21]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)


Epoch 1/20


c:\Users\Yatendra Pachori\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.3934 - loss: 1.1333 - val_accuracy: 0.4000 - val_loss: 1.0865
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.4415 - loss: 1.0745 - val_accuracy: 0.4000 - val_loss: 1.0706
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.4870 - loss: 1.0197 - val_accuracy: 0.4000 - val_loss: 1.0668
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.4623 - loss: 1.0494 - val_accuracy: 0.4000 - val_loss: 1.0664
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.4399 - loss: 1.0312 - val_accuracy: 0.4000 - val_loss: 1.0784
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.4481 - loss: 1.0265 - val_accuracy: 0.4000 - val_loss: 1.0837
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.4671 - loss: 0.9731 - val_accuracy: 0.4000 - val_loss: 1.0875
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.4614 - loss: 0.9549 - val_accuracy: 0.4000 - val_loss: 1.0862
Epo